# Variacional Univariate Gaussian

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import randn

In [2]:
%config InlineBackend.figure_format = "retina"

In this notebook we look at a factorized variational approximation to a Gaussian Distribution.

Let $\mathcal D= \{x_n\}_{n=1}^N$ i.i.d. random variables such that $x_n\sim \mathcal N(\mu, \tau^{-1})$. Our goal is to find a posterior distribution 

$$
    p(\mu, \tau | \mathcal D) \propto p(\tau)p(\mu|\tau) p(\mathcal D | \mu, \tau)
$$

With
* $p(\tau) = \text{Gam}(\tau|a_0, b_0)$
* $p(\mu|\tau) = \mathcal N(\mu|\mu_0, (\lambda_0\tau)^{-1})$
* $p(\mathcal D | \mu, \tau) = \prod_{n=1}^N \mathcal{N}(x_n | \mu, \tau^{-1})$

To find such posterior, we assume a factorized variational approximation to $p$ given by

$$
    q(\mu,\tau) = q^*_\mu(\mu) q^*_\tau(\tau)
$$


Since the optimum factors of a factorized variational approximation follow the relation

$$
    \log q^*_j({\bf z}_j) = \mathbb{E}_{m\neq j}[\log p({\bf z})] + C,
$$

It follows that

$$
    q^*_\mu(\mu) = \mathcal{N}\left(\mu \ \big | \frac{N\bar x + \mu_0}{N + \lambda_0}, [N + \lambda_0]\mathbb{E}[\tau]\right)
$$


$$
    q^*_\tau(\tau) = \text{Gam}\left(\tau \ \big | a_0 + \frac{N}{2}, b_0 + \frac{1}{2}\mathbb{E}\left[\lambda_0\tau(\mu - \mu_0)^2 + \sum_{n=1}^N(x_n - \mu)^2\right] \right)
$$